<a href="https://colab.research.google.com/github/jai2shan/TSAI_EVA6/blob/main/Session%204/Session_4_Less_than_20k_Parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading necessary packages

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

## Check for GPU 

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(1)
# defining batch size for training
batch_size = 1024

## Data loading and pre-processing

In [3]:
# Keyword arguments for train_loader and test _loader to load images to GPU if the available
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,   
                    transform=transforms.Compose([
                        transforms.ToTensor(), 
                        transforms.Normalize((0.1307,), (0.3081,)) 
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs) 


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

## Training and Testing - Definitions

In [4]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [15]:
dropout_value = 0.1

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value)
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 24

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 24
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(32),
            nn.Dropout(dropout_value)
        ) # output_size = 10
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=24, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(24),
            nn.Dropout(dropout_value)
        ) # output_size = 8
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=24, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(3, 3), padding=1, bias=False),         
            nn.BatchNorm2d(10)
        ) # output_size = 6
        
        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6)
        ) # output_size = 1

        self.dropout = nn.Dropout(dropout_value)


    def forward(self, img):
      img = self.convblock1(img)
      img = self.convblock2(img)
      img = self.convblock3(img)
      img = self.pool1(img)
      img = self.convblock4(img)
      img = self.convblock5(img)
      img = self.convblock6(img)
      img = self.convblock7(img)
      img = self.gap(img)        

      img = img.view(-1, 10)
      return F.log_softmax(img, dim=-1)

In [12]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 16, 24, 24]             256
        MaxPool2d-10           [-1, 16, 12, 12]               0
           Conv2d-11           [-1, 32, 10, 10]           4,608
             ReLU-12           [-1, 32, 10, 10]               0
      BatchNorm2d-13           [-1, 32, 10, 10]              64
          Dropout-14           [-1, 32,

In [14]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.95)
EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/59 [00:00<?, ?it/s]

EPOCH: 0


Loss=0.37634122371673584 Batch_id=58 Accuracy=79.23: 100%|██████████| 59/59 [00:07<00:00,  7.71it/s]
  0%|          | 0/59 [00:00<?, ?it/s]


Test set: Average loss: 0.3875, Accuracy: 9173/10000 (91.73%)

EPOCH: 1


Loss=0.12630324065685272 Batch_id=58 Accuracy=96.08: 100%|██████████| 59/59 [00:07<00:00,  7.69it/s]
  0%|          | 0/59 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0260, Accuracy: 9929/10000 (99.29%)

EPOCH: 18


Loss=0.04455804079771042 Batch_id=58 Accuracy=99.33: 100%|██████████| 59/59 [00:07<00:00,  7.42it/s]
  0%|          | 0/59 [00:00<?, ?it/s]


Test set: Average loss: 0.0238, Accuracy: 9935/10000 (99.35%)

EPOCH: 19


Loss=0.01888817921280861 Batch_id=58 Accuracy=99.32: 100%|██████████| 59/59 [00:07<00:00,  7.58it/s]



Test set: Average loss: 0.0299, Accuracy: 9915/10000 (99.15%)

